In [1]:
import numpy as np
import pandas as pd
import warnings

pd.set_option('display.max_rows', 1100)
pd.set_option('display.max_columns', 1100)
warnings.filterwarnings('ignore')

train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
cols = ["date", "lineName", "directionCode", "ampm"]
train_df["ampm"] = train_df["planArrival"].map(lambda x : "am" if int(x[:2]) <= 14 else "pm")

In [2]:
train_df["planArrival_int"] = train_df["planArrival"].map(lambda x : int(x.replace(":", "")))
train_df["y"] = train_df["delayTime"].values
train_df["delayTime"][(801 <= train_df["planArrival_int"]) & (train_df["planArrival_int"] <= 1400)] = np.nan
train_df["delayTime"][(1801 <= train_df["planArrival_int"])] = np.nan
train_df["isnanDelayTime"] = train_df["delayTime"].map(lambda x : 1 if x != x else 0)
mask = train_df["isnanDelayTime"] == 1

train_df["pred"] = train_df.groupby(["date", "trainNo"])["delayTime"].transform(lambda x: x.fillna(method="ffill"))
train_df["pred"] = train_df.groupby(cols)["pred"].transform(lambda x: x.fillna(x.mean()))

In [3]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(train_df[mask]["pred"], train_df[mask]["y"])
#benchmark1 1.33
#benchmark2 1.38

1.3858464247231885